In [ ]:
!pip install anvil-uplink

In [25]:
#please clone the provided webapp into your anvil profile
import anvil.server
import anvil.media 
anvil.server.connect("uplink key") #enter the uplink key for the cloned app
# instructions are provided in read me

Disconnecting from previous connection first...
Connecting to wss://anvil.works/uplink
Anvil websocket closed (code 1000, reason=b'')
Anvil websocket open
Fatal error from Anvil server: Incorrect uplink key
Anvil websocket closed (code 1000, reason=b'')


In [4]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2 
import numpy as np
import seaborn as sns
from IPython import get_ipython
import shutil
import glob
from tqdm import tqdm 
from PIL import Image
ipy = get_ipython()
if ipy is not None:
    ipy.run_line_magic('matplotlib', 'inline')

In [7]:
#connecting to drive since. no need if data is present locally
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
@anvil.server.callable
def get_path():
  path='/content/drive/MyDrive/final_project/'#give the path where you need the files to be uploaded
  return path

In [12]:
@anvil.server.callable
def create_folders(names):
  path=get_path()
  for name in names:
    if not os.path.exists(path+name):
      os.mkdir(path+name)
      os.mkdir(path+name+"/mean")
  return"successfully created"

In [11]:
@anvil.server.callable
def upload_images(image,name,i):
  path=get_path() #please enter the same path as above cell
  with anvil.media.TempFile(image) as img_file:
    img= Image.open(img_file)
    #exif = img.info['exif']
  #i=nanana.pop()
  image = img.save(path+name+"/image-"+str(i)+".jpg",quality=100)#exif=exif,quality=100)
  return "successfully uploaded"

In [10]:
@anvil.server.callable
def upload_check_img(image):
  path=get_path()
  with anvil.media.TempFile(image) as img_file:
    img= Image.open(img_file)
    #exif = img.info['exif']
  image = img.save(path+"/check_img/check.jpg",quality=100)#exif=exif)
  return "successfully uploaded"


In [13]:
@anvil.server.callable
def std_edge_img(train_img):
  x_train = np.array(train_img, np.float32)
  std_img = np.std(x_train, axis=0)
  edges = cv2.Canny(np.uint8(std_img),209,210)
  blackto3d = cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
  blackto3d[edges==0] = (0,0,0)
  return blackto3d

In [15]:
@anvil.server.callable
def canny_edge(image):
  img2=cv2.Canny(np.uint8(image),209,210)
  blacktogreen2 = cv2.cvtColor(img2, cv2.COLOR_GRAY2BGR)
  blacktogreen2[img2==0] = (10,255,10)
  return blacktogreen2

In [16]:
@anvil.server.callable
def mean_edge_img(train_img):
  x_train = np.array(train_img, np.float32)
  mean_img_l = np.mean(x_train, axis=0)
  img2=canny_edge(mean_img_l)
  return img2


In [17]:
def ret_limit(image1,image2):

  res = cv2.absdiff(image1, image2)

  res = res.astype(np.uint8)

  percentage = (np.count_nonzero(res) * 100)/ res.size
  
  return percentage

In [19]:
@anvil.server.callable
def create_mean(name):
#creating the final mean image for the uploaded images
  path=get_path()
  train_img = []
  mean_img=''
  mean_image_pics=[]
  for img_path in tqdm(glob.glob(path+name+"/*.jpg")):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (512,512))
    train_img.append(img)
  train_img = np.array(train_img, np.float32)

  image1 = np.asarray(std_edge_img(train_img), np.float64)
  image2 = np.asarray(mean_edge_img(train_img), np.float64)

  limit = 100-ret_limit(image1,image2)
  print(limit)
  for image in tqdm(train_img):
    image2 = np.asarray(canny_edge(image), np.float64)
    x =100-ret_limit(image1,image2)
    if x<=limit:
      print(x)
      mean_image_pics.append(image)

  mean_img = np.mean(mean_image_pics, axis=0)
  #saving as npy, since its a numpy matrix
  np.save(path+name+"/mean/"+name+"-mean.npy", mean_img)    # .npy extension is added if not given
  cv2.imwrite("/content/mean.jpg",mean_img,[cv2.IMWRITE_JPEG_QUALITY, 100])
  return "done"


In [21]:
@anvil.server.callable
def show_animal(name):
  black_img = np.zeros((512, 512,3), dtype = "uint8")
  path=get_path()
  ret=[]
  image_mean=np.load(path+name+"/mean/"+name+"-mean.npy")
  imgs=[]
  paths=[]
  ret_list=[]
  for img_path in tqdm(glob.glob(path+name+"/*.jpg")):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (512,512))
    imgs.append(img)
    paths.append(img_path)
  #imgs = np.array(imgs, np.float32)
  image1=np.asarray(image_mean,np.float64)
  i=0
  count=0
  for image in imgs:
    image2=np.asarray(image,np.float64)
    res=cv2.absdiff(image1,image2)
    res = res.astype(np.uint8)
    result = cv2.threshold(res, 0, 25.5, cv2.THRESH_BINARY)[1]
    x=100-ret_limit(black_img,result)
    if (x>5.5):
      count+=1
    if x>5.5and x<9:
      g = anvil.media.from_file(paths[i], 'image/jpeg')
      ret.append(g)
      print('image-'+str(i+1)+'.jpg')
    i+=1
  print(count)
  return ret


In [23]:
#checks to see where the new image belongs
@anvil.server.callable
def check_upload(names):
  path=get_path()
  black_img = np.zeros((512, 512,3), dtype = "uint8")
  ret = "  image doesnt belong to any location"
  img = cv2.imread(path+"/check_img/check.jpg")
  img = cv2.resize(img, (512,512))
  for name in names:
    image_mean=np.load(path+name+"/mean/"+name+"-mean.npy")
    image1=np.asarray(image_mean,np.float64)
    image2=np.asarray(img,np.float64)
    res=cv2.absdiff(image1,image2)
    res = res.astype(np.uint8)
    result = cv2.threshold(res, 0,25.5, cv2.THRESH_BINARY)[1]
    x=100-ret_limit(black_img,result)
    print(x)
    if x>5.5:
      ret="  image belongs to "+name
      break
  return ret

